In [1]:
import os, pandas as pd, numpy as np, re, time
from functools import reduce
pd.set_option('display.max_columns', 1500)

#def main():
sage = '/home/wraikes/Dropbox/partnership/dmt/data/sage_not_final'
#sage = r'C:\Users\williamr.PDFK\Dropbox\partnership\dmt_temp\Sage (Mobile Phone) Data 8.31.17'
os.chdir(sage)
files_to_exclude = ['digital-marshmallow-status_8.31.17.csv',
                    'digital-marshmallow-appVersion_8.31.17.csv']
end_string = '_8.31.17.csv'

In [2]:
def main():
    sage = '/home/wraikes/Dropbox/partnership/dmt/data/sage_not_final'
    #sage = r'C:\Users\williamr.PDFK\Dropbox\partnership\dmt_temp\Sage (Mobile Phone) Data 8.31.17'
    os.chdir(sage)
    
    files_to_exclude = ['digital-marshmallow-status_8.31.17.csv',
                        'digital-marshmallow-appVersion_8.31.17.csv']
    end_string = '_8.31.17.csv'
    bl = 'baseline'
    _21 = '21-day-assessment'
    
    common_df = clean_common_dfs(os.listdir(), files_to_exclude, end_string)
    
    #To target the behavior_choices_1 df
    common_df = selected_recode(common_df)
        
    bart_att = [
        [bl, 'BART0.25'],
        [bl, 'BART250.00'],
        [_21, 'BART0.25'],
        [_21, 'BART250.00']       
    ]
    
    bart_df = clean_df('digital-marshmallow-bart-v4_8.31.17.csv',
                   end_string,
                   bart_att,
                   var_2='data.json.variable_label')
    
    delay_att = [
        [bl, 'dd_time_6_months'],
        [bl, 'dd_money_6_month'],
        [bl, 'dd_money_1_month'],
        [bl, 'dd_time_1_year'],
        [_21, 'dd_time_6_month'],
        [_21, 'dd_money_6_month'],
        [_21, 'dd_money_1_month'],
        [_21, 'dd_time_1_year']            
    ]
    
    delay_df = clean_df('digital-marshmallow-delay_discounting_raw-v6_8.31.17.csv',
                    end_string,
                    delay_att,
                    var_2='data.json.variableLabel')    
    
    delay_df = df_array_last_value(delay_df)
    delay_df = df_split_array(delay_df)
    
    behave_4_att = [
        [bl]
    ]
    
    behave_4_df = clean_df('digital-marshmallow-behavior_choices_4_bl-v2_8.31.17.csv',
                       end_string,
                       behave_4_att)
    
    behave_4_df = selected_spread(behave_4_df)
    
    discount_att = [
        [bl, 'pd_constant_money'],
        [bl, 'pd_constant_probabiliy'],
        [_21, 'pd_constant_money'],
        [_21, 'pd_constant_probability']
    ]
    
    discount_df = clean_df('digital-marshmallow-discounting_raw-v2_8.31.17.csv',
                      end_string,
                      discount_att,
                      var_2='data.json.variableLabel')
    
    evening_note_att = [
        [bl]
    ] 
    
    evening_note_df = clean_df('digital-marshmallow-evening_notification_time-v2_8.31.17.csv', 
                           end_string,
                           evening_note_att) 
    
    gonogo_att = [
        [bl, 'go_no_go_stable_stimulus_active_task'],
        [bl, 'go_no_go_variable_stimulus_active_task'],
        [_21, 'go_no_go_stable_stimulus_active_task'],
        [_21, 'go_no_go_variable_stimulus_active_task']
    ]
    
    gonogo_df = clean_df('digital-marshmallow-goNoGo-v2_8.31.17.csv', 
                     end_string,
                     gonogo_att,
                     var_2='data.json.variable_label')
    
    morning_note_att = [
        [bl]
    ]   
    
    morning_note_df = clean_df('digital-marshmallow-morning_notification_time-v3_8.31.17.csv',
                          end_string,
                          morning_note_att)
    
    pam_mult_att = [
        [bl],
        [_21]
    ]
     
    pam_mult_df = clean_df('digital-marshmallow-pam_multiple-v2_8.31.17.csv',
                      end_string,
                      pam_mult_att)
    
    df = merge_dfs([common_df, 
                        bart_df,
                        delay_df,
                        behave_4_df,
                        discount_df,
                        evening_note_df,
                        gonogo_df,
                        morning_note_df,
                        pam_mult_df])  

    df = timestamp_conv(df)
    
    return df

In [3]:
def merge_dfs(dfs):
    df = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                             on='externalId'), 
                dfs)
    
    return df

In [4]:
def clean_common_dfs(directory, exclude_files, string):
    '''
    Go through and process all files in directory that don't need columns "spread".
    If extra processing is needed, a different function will be used.
    Return a single dataframe, of all data merged.
    '''
    
    new_dfs = []
    
    for file in directory: 
        if file not in exclude_files:
            df_name = create_df_name(file, string)
            df = pd.read_csv(file)
            df = remove_dupes(df)
            if df_name in ('demographics-v2', 'generally_sem_diff_bl-v2'):
                df = clean_df_dupe(df, 'yQ7pYy')
            
            if len(df.externalId) == len(df.externalId.unique()):
                df = col_relabel(df, df_name)
                new_dfs.append(df)
    
    return merge_dfs(new_dfs)

In [5]:
def create_df_name(string, end_string):
    string = string.replace('digital-marshmallow-', '')
    string = string.replace(end_string, '')
    
    return string 

In [6]:
def col_relabel(df, prefix, att=None):
    '''
    Remove: 'metadata.json.' and 'data.json.'.
    Replace spaces with underscores.
    Append new name to existing column name, except if externalId.
    '''
    new_cols = []
    col_re = re.compile('metadata.json.|data.json.')
    
    for col in df.columns:
        new_col = re.sub(col_re, '', col)
    
        if new_col != 'externalId':
            if att and len(att) == 2:
                new_col = 'SAGE_{}_{}_{}___{}'.format(prefix, 
                                                      att[0], 
                                                      att[1], 
                                                      new_col
                                                     )
            elif att and len(att) == 1:
                new_col = 'SAGE_{}_{}___{}'.format(prefix, 
                                                   att[0], 
                                                   new_col
                                                  )                
            
            else:
                new_col = 'SAGE_{}___{}'.format(prefix, 
                                                new_col
                                                ) 
                                              
        new_col = new_col.replace(' ', '_')
        new_col = new_col.replace('-', '_')
        new_cols.append(new_col)
    
    df.columns = new_cols
    
    return df

In [7]:
def remove_dupes(df):
    
    test_users = [
        'ThpMV2Achc', 
        'SEkQVTCe6j', 
        'Wh8NSX3DHL', 
        'SaXFr2kPZa', 
        'VWUcSp4TeH', 
        'yXEfAmW682', 
        'gwEpQR8j9B',
        'WbbNWM4RAF', 
        'D5bzYrfd8E', 
        'LJcmEFWp74', 
        'ULoF3MM1nN',
        'G9Bbsg'
    ]

    diff_study = [
        'rL8eA3',
        'rLg5xs',
        'rLrD9h',
        'rLP7H2',
        'rL6s6h',
        'aOyzBg',
        'aORA43',
        'aOh48U',
        'aOLu4K',
        'aOQtxv',
        'aO5TvQ',
        'mPC9S8',
        'mPgquX',
        'mP5xkB',
        'mPSQvh',
        'mPYk2p',
        'mP3rbd'
    ]

    
    return df[~df.externalId.isin(test_users + diff_study)]

In [8]:
def spread_cols(df, df_name, attributes, var_2):
    '''
    Merge the dataframes into one, with new columns.
    '''
    dfs = []
    var_1 = 'metadata.json.taskIdentifier'
    
    for att in attributes:
        cols = df.columns
        new_df = pd.DataFrame(columns=cols)
        
        for ix, row in df.iterrows():
            if var_2:
                if row[var_1] == att[0] and row[var_2] == att[1]:
                    new_df = new_df.append(row, ignore_index=True)
            else:
                if row[var_1] == att[0]:
                    new_df = new_df.append(row, ignore_index=True)
                    
        new_df = col_relabel(new_df, df_name, att)
        dfs.append(new_df)
    
    return merge_dfs(dfs)

In [9]:
def clean_df(file, string, attributes, var_2=None):
    df_name, df = create_df_name(file, string), pd.read_csv(file)
    df = remove_dupes(df)
    
    if df_name == 'goNoGo-v2':
        df = clean_df_dupe(df, 'ksJM3Y')
    
    if df_name == 'behavior_choices_4_bl-v2':
        df = clean_df_dupe(df, 'yQ7pYy')
    
    df = spread_cols(df, df_name, attributes, var_2)
    
    return df

In [10]:
def clean_df_dupe(df, _id):
   
    indices = df.index[df.externalId == _id]
    df.drop(indices[-1], axis=0, inplace=True)
    
    return df

In [11]:
def timestamp_conv(df):
    
    for col in df.columns:
        if any([col.endswith(x) for x in ('createdOn', 'startDate', 'endDate')]):
            df[col] = df[col][df[col].notnull()].apply(lambda x: time.strftime("%m.%d.%Y %H:%M - %Z", time.localtime(x / 1000)))
    
    return df

In [12]:
def array_split(value, i):
    if pd.notnull(value):
        return list(map(float, value[1:-1].split(',')))[i]
    else:
        return value

def df_array_last_value(df):
    new_df = df.copy()

    for col in df.columns:
        if 'nowArray' in col:
            new_col = col + '_last'
            new_df[new_col] = df[col].apply(lambda x: array_split(x, -1))
    
    return new_df

def df_split_array(df):
    new_df = df.copy()
    
    for col in df.columns:
        if all([x in col for x in ('times', 'delay', 'SAGE')]):
            for i in range(6):
                new_col = col + '_v' + str(i)
                new_df[new_col] = df[col].apply(lambda x: array_split(x, i))
                
    return new_df

In [13]:
def selected_spread(df):
    #Note: codebook is incorrect. 'gamble' vs. 'gambling'.
    terms = ['junkfood', 'gambling', 'smoke', 'mobile', 'overeat', 'mast', 'cut', 'smedia',
             'alcohol', 'porn', 'hair', 'gaming', 'sitting', 'spend', 'temper', 'othermore']

    for term in terms:
        col_name = 'SAGE_behavior_choices_4_bl_v2_baseline___selected' + '_' + term
        df[col_name] = np.where(df['SAGE_behavior_choices_4_bl_v2_baseline___selected'].str.contains(term), 1, 0)
        
    return df

def recode(value):
    
    terms = ['junkfood', 'gambling', 'smoke', 'mobile', 'overeat', 'mast', 'cut', 'smedia',
             'alcohol', 'porn', 'hair', 'gaming', 'sitting', 'spend', 'temper', 'othermore']
    
    for ix, term in enumerate(terms):
        if term in value:
            return ix 

def selected_recode(df):
    
    df['SAGE_behavior_choices_1_bl_v1___selected_recode'] = df['SAGE_behavior_choices_1_bl_v1___selected'].apply(recode)
    
    return df

In [14]:
test = main()

In [19]:
test.tail(15)

SAGE_generally_sem_diff_bl_v2___ROW_ID  \
62                                      77   
63                                      78   
64                                      79   
65                                      80   
66                                      81   
67                                      83   
68                                      84   
69                                      85   
70                                      86   
71                                      87   
72                                      89   
73                                      94   
74                                      96   
75                                      97   
76                                     101   

    SAGE_generally_sem_diff_bl_v2___ROW_VERSION  \
62                                           35   
63                                           36   
64                                           36   
65                                           37   
66                                           37   
67                                           39   
68                                           39   
69                                           39   
70                                           40   
71                                           41   
72                                           41   
73                                           42   
74                                           43   
75                                           43   
76                                           44   

   SAGE_generally_sem_diff_bl_v2___recordId  \
62     0e050199-4db1-4b5d-9e7f-6b3de6079fa8   
63     ce0aae0b-75de-45f8-a5f6-fd768f9eb829   
64     0e23f803-0892-42e8-946d-d9e9ce67ce02   
65     5b5d8a13-0c23-4fe8-aa79-3fd00c63e9c9   
66     25b2980e-c902-4ea2-9581-b7d49653ee67   
67     e3f274b9-8b62-4753-9192-22ef38e0f83a   
68     f78756a9-ced5-4991-9a39-252d69e27dfe   
69     485bed95-7394-432f-8ba5-8bda3992136a   
70     2b8511af-3ee4-4da4-86fd-053ade67dc48   
71     8e26c064-8127-4336-a291-a3da4ce0e5ff   
72     7eff74eb-5881-4875-a210-0b4101f2b203   
73     267ee72d-47eb-4713-8656-0a11fdc8e810   
74     be77170c-05c7-4e54-a580-47d307dbe4f1   
75     02baec66-ee8f-4b01-975b-fc23ce3850db   
76     fe885303-3a4c-40d0-ae92-5cb0aa4ddfb6   

   SAGE_generally_sem_diff_bl_v2___appVersion  \
62                      version 1.2, build 18   
63                      version 1.2, build 18   
64                      version 1.2, build 18   
65                      version 1.2, build 18   
66                     version 1.0.2, build 1   
67                      version 1.2, build 18   
68                      version 1.2, build 18   
69                      version 1.2, build 18   
70                      version 1.2, build 18   
71                      version 1.2, build 18   
72                      version 1.2, build 18   
73                      version 1.2, build 18   
74                     version 1.0.2, build 1   
75                     version 1.0.2, build 1   
76                      version 1.2, build 18   

   SAGE_generally_sem_diff_bl_v2___phoneInfo  \
62                     iPhone 5S; iOS/10.3.3   
63                      iPhone 6; iOS/10.3.2   
64                     iPhone 6+; iOS/10.3.2   
65                      iPhone 6; iOS/10.3.2   
66                          Samsung SM-G900T   
67                     iPhone 6S; iOS/10.3.3   
68                      iPhone 6; iOS/10.3.3   
69                     iPhone 5S; iOS/10.3.3   
70                      iPhone 6; iOS/10.3.3   
71                    iPhone 6S+; iOS/10.3.3   
72                     iPhone 7+; iOS/10.3.3   
73                      iPhone 6; iOS/10.3.3   
74                       Motorola Moto G (4)   
75                  Samsung SAMSUNG-SM-G900A   
76                     iPhone 6S; iOS/10.3.3   

   SAGE_generally_sem_diff_bl_v2___uploadDate  \
62                                  7/27/2017   
63                                  7/29/2017